In [2]:
import pymongo
client = pymongo.MongoClient('mongodb://localhost:27017/')
jobs_db = client['jobs']
table = jobs_db['extracted jobs wnh']

In [3]:
all_data = list(table.find())
print(all_data[-1])

{'location': '/WorkProjects/ProjectDetail/PHP-Mysql-Coders-for-Shopping-Marketplace-site-development/78622', 'title': 'PHP/Mysql Coders for Shopping - Marketplace site development', '_id': ObjectId('59a16700dc9cb52080a76632'), 'details': 'PHP/Mysql Coders for Shopping - Marketplace site development\r\n* Coder should be available online from 9AM to 8PM IST\r\n* Coder should have experience in Codeigniter framework\r\n* Coder should have done API integrations \r\n* Shopping cart development experience - preferred\r\n\r\nThe bid should be for Flat Monthly pay if work is good this may be an ongoing job. We need experienced people only and one who can dedicate this time.  We will drop any one who is doing multiple projects in our work time.\r\n\r\nA message without the Bid amount is ignored.  We will take your interview by phone or Skype so please provide both the details. Should be ready to start immediately when hired.', 'skills': '\nSkills:\nPHP, MySQL, CodeIgniter', 'date posted': '2017

In [4]:
import re
import operator


def is_number(s):
    try:
        float(s) if '.' in s else int(s)
        return True
    except ValueError:
        return False


def load_stop_words(stop_word_file):
    """
    Utility function to load stop words from a file and return as a list of words
    @param stop_word_file Path and file name of a file containing stop words.
    @return list A list of stop words.
    """
    stop_words = []
    for line in open(stop_word_file):
        if line.strip()[0:1] != "#":
            for word in line.split():  # in case more than one per line
                stop_words.append(word)
    return stop_words


def separate_words(text, min_word_return_size):
    """
    Utility function to return a list of all words that are have a length greater than a specified number of characters.
    @param text The text that must be split in to words.
    @param min_word_return_size The minimum no of characters a word must have to be included.
    """
    splitter = re.compile('[^a-zA-Z0-9_\\+\\-/]')
    words = []
    for single_word in splitter.split(text):
        current_word = single_word.strip().lower()
        # leave numbers in phrase, but don't count as words, since they tend to invalidate scores of their phrases
        if len(current_word) > min_word_return_size and current_word != '' and not is_number(current_word):
            words.append(current_word)
    return words


def split_sentences(text):
    """
    Utility function to return a list of sentences.
    @param text The text that must be split in to sentences.
    """
    sentence_delimiters = re.compile(u'[.!?,;:\t\\\\"\\(\\)\\\'\u2019\u2013]|\\s\\-\\s')
    sentences = sentence_delimiters.split(text)
    return sentences


def build_stop_word_regex(stop_word_file_path):
    stop_word_list = load_stop_words(stop_word_file_path)
    stop_word_regex_list = []
    for word in stop_word_list:
        word_regex = r'\b' + word + r'(?![\w-])'
        stop_word_regex_list.append(word_regex)
    stop_word_pattern = re.compile('|'.join(stop_word_regex_list), re.IGNORECASE)
    return stop_word_pattern


def generate_candidate_keywords(sentence_list, stopword_pattern):
    phrase_list = []
    for s in sentence_list:
        tmp = re.sub(stopword_pattern, '|', s.strip())
        phrases = tmp.split("|")
        for phrase in phrases:
            phrase = phrase.strip().lower()
            if phrase != "":
                phrase_list.append(phrase)
    return phrase_list


def calculate_word_scores(phraseList):
    word_frequency = {}
    word_degree = {}
    for phrase in phraseList:
        word_list = separate_words(phrase, 0)
        word_list_length = len(word_list)
        word_list_degree = word_list_length - 1
        for word in word_list:
            word_frequency.setdefault(word, 0)
            word_frequency[word] += 1
            word_degree.setdefault(word, 0)
            word_degree[word] += word_list_degree
    for item in word_frequency:
        word_degree[item] = word_degree[item] + word_frequency[item]

    # Calculate Word scores = deg(w)/frew(w)
    word_score = {}
    for item in word_frequency:
        word_score.setdefault(item, 0)
        word_score[item] = word_degree[item] / (word_frequency[item] * 1.0)
    return word_score


def generate_candidate_keyword_scores(phrase_list, word_score):
    keyword_candidates = {}
    for phrase in phrase_list:
        keyword_candidates.setdefault(phrase, 0)
        word_list = separate_words(phrase, 0)
        candidate_score = 0
        for word in word_list:
            candidate_score += word_score[word]
        keyword_candidates[phrase] = candidate_score
    return keyword_candidates


class Rake(object):
    def __init__(self, stop_words_path):
        self.stop_words_path = stop_words_path
        self.__stop_words_pattern = build_stop_word_regex(stop_words_path)

    def run(self, text):
        sentence_list = split_sentences(text)

        phrase_list = generate_candidate_keywords(sentence_list, self.__stop_words_pattern)

        word_scores = calculate_word_scores(phrase_list)

        keyword_candidates = generate_candidate_keyword_scores(phrase_list, word_scores)

        sorted_keywords = sorted(keyword_candidates.items(), key=operator.itemgetter(1), reverse=True)
        return sorted_keywords

In [5]:
import os
v = Rake(os.getcwd()+'\\analytics\Stop_list.txt')

In [6]:
v.run(all_data[-1]['details'])

[('api integrations \r\n* shopping cart development experience', 30.0),
 ('marketplace site development\r\n* coder', 16.333333333333332),
 ('8pm ist\r\n* coder', 9.333333333333334),
 ('codeigniter framework\r\n* coder', 9.333333333333334),
 ('flat monthly pay', 9.0),
 ('start immediately', 4.0),
 ('ongoing job', 4.0),
 ('multiple projects', 4.0),
 ('php/mysql coders', 4.0),
 ('experienced people', 4.0),
 ('shopping', 3.5),
 ('bid amount', 3.5),
 ('experience', 3.5),
 ('work time', 3.0),
 ('work', 1.5),
 ('bid', 1.5),
 ('time', 1.5),
 ('online', 1.0),
 ('ready', 1.0),
 ('details', 1.0),
 ('skype', 1.0),
 ('message', 1.0),
 ('good', 1.0),
 ('dedicate', 1.0),
 ('drop', 1.0),
 ('interview', 1.0),
 ('phone', 1.0),
 ('preferred', 1.0),
 ('hired', 1.0),
 ('9am', 1.0),
 ('provide', 1.0)]

In [8]:
predicted_data = all_data[-7:-1]

import time

for i in all_data:
    r = i['details']
    predicted = v.run(r)
    print(predicted[0:5])
    time.sleep(9)

[('writer status', 4.0), ('short notice', 4.0), ('part timer', 4.0), ('15days basis', 4.0), ('part-time writer', 4.0)]


[('school erp product', 9.0), ('commission basis', 4.0), ('recurring commission', 4.0), ('guys', 1.0), ('marketing', 1.0)]


[('organising references', 4.0), ('presence required', 4.0), ('excel file', 4.0), ('work researching', 4.0), ('related field', 4.0)]


[('/affiliate/subscribe/ajax-complete-13-e-books-set-2e67b/\r\n\r\njava spring struct jsp 45 e-books set 299- inr\r\nhttps', 89.66666666666667), ('affiliate marketers apply\r\n\r\najax complete 13 e-books set 299- inr\r\nhttps', 86.16666666666667), ('/affiliate/subscribe/oracle-and-sql-32-e-books-set-d57dc/\r\n\r\nado dot net 10 e-books set 299- inr\r\nhttps', 75.66666666666666), ('/affiliate/subscribe/web-services-5-e-books-set-1a3b1/\r\n\r\nasp dot net 96 e-books set 499- inr\r\nhttps', 75.0), ('/affiliate/subscribe/c-10-e-books-set-list/\r\n\r\nweb services 5 e-books set 199- inr\r\nhttps', 62.0)]


[('/affiliate/subscribe/ajax-complete-13-e-books-set-2e67b/\r\n\r\njava spring struct jsp 45 e-books set 299- inr\r\nhttps', 89.66666666666667), ('affiliate marketers apply\r\n\r\najax complete 13 e-books set 299- inr\r\nhttps', 86.16666666666667), ('/affiliate/subscribe/oracle-and-sql-32-e-books-set-d57dc/\r\n\r\nado dot net 10 e-books set 299- inr\r\nhttps', 75.66666666666666), ('/affiliate/subscribe/web-services-5-e-books-set-1a3b1/\r\n\r\nasp dot net 96 e-books set 499- inr\r\nhttps', 75.0), ('/affiliate/subscribe/c-10-e-books-set-list/\r\n\r\nweb services 5 e-books set 199- inr\r\nhttps', 62.0)]


...

[('manoj shinh- don', 9.0), ('fund escrow', 4.0), ('paypal payment', 4.0), ('cheated', 1.0), ('work', 1.0)]


[('writing fitness product reviews', 10.0), ('dedicated freelance writer', 8.5), ('sample 500 word article', 8.25), ('good quality article', 7.75), ('fitness niche', 5.0)]


[('startup company', 4.0), ('business plan', 4.0), ('write', 1.0)]


[('apa 6th edition referencing style', 25.0), ('econ6001 economic principles\r\n\r\n\r\ncontext', 16.0), ('theory underlining graph 2 differ', 13.714285714285714), ('graph 1identify dependent variable', 12.714285714285715), ('source iscorrectly attributed', 8.0)]


KeyboardInterrupt: 

NameError: name 'all_data' is not defined

this is cleaned

...
Hi, I need FreeLancer/ FullTime  Web Designer / UI designer for website and developmenet work for my software. Preferably from Hyderabad. The software is built on PHP/Codeignitor. I have to add some new  modules and change the existing design. I want the web developer to sync his ideas  on UI/Design with my PhP developer .My website is www.healthfox.com. 

this is 
